### Aruco sensor processing

Experimentation for the development of an Aruco tag based sensor processing system. 

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import numpy as np
import cv2
import torch
import pprint

from sensorprocessing import sp_aruco

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sensorprocessing.aruco.utils import aruco_display

arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
arucoParams = cv2.aruco.DetectorParameters()

In [ ]:
# Specify the experiment
experiment = "sensorprocessing_aruco"
run = "aruco_128" 
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)
sp = sp_aruco.ArucoSensorProcessing(exp, device)

In [ ]:
camera_number = Config().values["robot"]["active_camera_list"][0]
print(camera_number)
#camera_number = 2
cap = cv2.VideoCapture(camera_number)

while(True):
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    frame = cv2.resize(frame, (256, 256))
    # imgresized = frame

    print(frame)

    #z = sp.process(frame)
    #print(z)

    # insert ARUCO code here

    marker_corners, marker_ids, rejected_candidates = cv2.aruco.detectMarkers(
        frame, arucoDict, parameters=arucoParams)
    
    detected_markers = aruco_display(marker_corners, marker_ids, rejected_candidates, frame)

    # end of ARUCO code

    # cv2.imshow('Press q to exit', frame)
    cv2.imshow('Press q to exit', detected_markers)
    ret = cv2.waitKey(1)
    if ret != -1:
        ret2 = ret & 0xFF
        if ret2 == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(marker_corners)

In [ ]:
marker_ids

In [ ]:
frame.shape

This code creates the encoding for the Aruco markers because 

In [ ]:
MARKER_COUNT = 10
XMAX = float(frame.shape[1])
YMAX = float(frame.shape[0])
NORMALIZER = np.tile([XMAX, YMAX], 4)

z = np.ones(MARKER_COUNT * (8 + 1)) * -1.0
for id, corners in zip(marker_ids, marker_corners):
    detection = corners[0].flatten() / NORMALIZER
    idn = id[0]
    z[idn * (8+1):(idn+1) * (8+1)-1] = detection


In [ ]:
z[0:9]